In [1]:
from sklearn import svm

In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split

In [3]:
df1 = pd.read_excel('data-size56.xlsx', sheet_name=0)

In [4]:
print(df1.columns)
print(df1.shape)

Index(['f ', 'd', 'T', 'Stress', 'Strain'], dtype='object')
(56, 5)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df1.loc[:,['f ', 'd', 'T']], \
df1.loc[:, 'Stress'], train_size = 51, shuffle = False)

In [6]:
from sklearn.cross_decomposition import PLSSVD
# from sklearn import ensemble
model = PLSSVD(n_components=1)
# model = KNeighborsRegressor()
# model = make_pipeline(StandardScaler(), LinearSVR(dual="auto", random_state=0, tol=1e-5))
# model = BernoulliRBM(n_components=2)
# model.fit(X_train, y_train)
model.fit(X_train.to_numpy(), y_train.to_numpy())
y_pred = model.predict(X_test)

AttributeError: 'PLSSVD' object has no attribute 'predict'

In [ ]:
import numpy as np
print(type(y_pred))
print(y_pred.shape)
y_pred = np.concatenate((model.predict(X_train), y_pred), axis=0)
print(y_pred.shape)
#y_pred = model.predict(df1.loc[:, ['Extrusion Temperature (in C)', 'Layer Height (mm)', 'Shell Thickness (mm)']].to_numpy())

In [ ]:
result = df1.loc[:, ['f ', 'd', 'T', 'Stress']].reset_index(drop=True)
# result = df1.loc[:, ['Dataset Type', 'Expt', 'XGBR']].reset_index(drop=True)
result['y_pred'] = pd.Series(y_pred)
result

In [ ]:
# print("r2 score (author) : ", r2_score(result['Stress'].to_numpy(), result['XGBR'].to_numpy()))
print("r2 score (amt) : ", r2_score(result['Stress'].to_numpy(), result['y_pred'].to_numpy()))
print("MAE (amt) : ", mean_squared_error(result['Stress'].to_numpy(), result['y_pred'].to_numpy()))
#print(pow(mean_squared_error(y_test.to_numpy(), y_pred), 0.5))


In [ ]:
# Rewriting r2score definiton
def getRSquared(y, y_pred):
    ssr = 0
    for i in range(len(y)):
        ssr = ssr + pow(y[i] - y_pred[i], 2)
    sst = 0
    ymean = 0
    for y_i in y:
        ymean = ymean + y_i
    ymean = ymean / len(y)
    for i in range(len(y)):
        sst = sst + pow(y[i] - ymean, 2)
    return 1 - ssr / sst